# Training model

In [ ]:
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

# # Read in example image and get shape
# img = load_img(trainPath + "/Quince/0_100.jpg")
# plt.imshow(img)
# plt.show()

# imgA = img_to_array(img)
# print(imgA.shape)        
# Build model
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(100,100,3)))
model.add(MaxPooling2D())
model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(5000, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(131, activation="softmax"))

print(model.summary())


# Compile model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

stepsPerEpoch = np.ceil(train_generator.samples / batchSize)
validationSteps = np.ceil(test_generator.samples / batchSize)

# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = model.fit(train_generator, steps_per_epoch=stepsPerEpoch, epochs=5, validation_data=test_generator, validation_steps=validationSteps, callbacks=[stop_early])

model.save("/kaggle/working/fruits360.h5")     # Add file path to save the model to

# Testing model

In [ ]:
!pip uninstall opencv-python-headless -y 
!pip install opencv-python --upgrade
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv

# Load model
model = tf.keras.models.load_model("/kaggle/working/fruits360.h5")       # Put in model path here
print(model.summary())

# Load categories
source_folder = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test"
categories = os.listdir(source_folder)
# categories = categories.sort()
print(categories)
print(len(categories))      # Should be 131

# Load and prepare image
def prepareImage(path):
    img = load_img(path, target_size=(100,100))
    imgArray = img_to_array(img)
    # print(imgArray.shape)
    imgArray = np.expand_dims(imgArray, axis=0)
    imgArray = imgArray / 255.
    return imgArray

testImgPath = "/kaggle/input/eggplant/eggplant.jpg"
imageForModel = prepareImage(testImgPath)

resultArray = model.predict(imageForModel, verbose=1)
answers = np.argmax(resultArray, axis=1)
print(categories[answers[0]])

# Show image with text below
img = cv.imread(testImgPath)
cv.putText(img, categories[answers[0]],(0,50), cv.FONT_HERSHEY_COMPLEX, 1, (209, 19, 77), 2)
cv.imshow('img', img)
cv.waitKey(0)




# Hyperparameter Tuning

In [ ]:
!pip install tensorflow-gpu==2.10
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline.py')

# Hyperparameter Tuning 2

In [ ]:
import keras_tuner as kt
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

#class MyHyperModel(kt.HyperModel):
def build(hp):
    model = Sequential()
    model.add(Conv2D(filters=128, kernel_size=3, activation=hp.Choice('activation', ['relu', 'PReLU', 'elu']), input_shape=(100,100,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(rate=hp.Float('rate',min_value=0.3,max_value=0.8,step=0.1)))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('units',3000,8000,500), activation=hp.Choice('activation_2', ['relu', 'PReLU', 'elu'])))
    model.add(Dense(units=hp.Int('units_2',500,1500,250), activation="relu"))
    model.add(Dense(131, activation="softmax"))
    #model.add(keras.layers.Dense(
     #   hp.Choice('units', [8, 16, 32]),
      #  activation='relu'))
   # model.add(keras.layers.Dense(1, activation='relu'))
    model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])
    
    print(model.summary())
    return model

#def fit(hp, model):
    # Early stopping
#stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

#history = model.fit(train_generator, steps_per_epoch=stepsPerEpoch, epochs=5, validation_data=test_generator, validation_steps=validationSteps, callbacks=[stop_early])


trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")
#x = MyHyperMode(kt.HyperModel)
#build(kt.HyperParameters())
#tuner = kt.Hyperband(hypermodel = build, objective="val_accuracy", max_epochs=10, hyperband_iterations=10,seed=1,max_retries_per_trial=0, max_consecutive_failed_trials=3)
tuner = kt.RandomSearch(hypermodel = build, objective="val_accuracy", max_trials=10,executions_per_trial=1,overwrite=False)
#tuner.search_space_summary()
tuner.search(train_generator,epochs=2,validation_data=test_generator)
tuner.results_summary()

# VGG 16

In [21]:
import tensorflow as tf
import os
from keras.models import Model
from keras.utils import img_to_array, load_img
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
#from livelossplot.inputs.keras import PlotLossesCallback
import numpy as np

#import cv2 as cv
#from keras.models import Sequential
#from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
#import matplotlib.pyplot as plt


BATCH_SIZE = 64

'''train_datagen = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing
'''

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
            
    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(5000, activation='relu')(top_model)
    top_model = Dense(1000, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print(model.summary())
    return model

input_shape = (100, 100, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=131

n_steps = train_generator.samples // batchSize
n_val_steps = test_generator.samples // batchSize
n_epochs = 50

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)


# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = vgg_model.fit(train_generator, steps_per_epoch=n_steps, epochs=5, validation_data=test_generator, validation_steps=n_val_steps, callbacks=[stop_early])


Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.
Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                  